In [1]:
import cv2
import numpy as np
import os
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
from sklearn import *

vocab_size_percentage = 14
test_size = 0.1814186208012823
n_neighbors = 3

In [2]:
class Preprocessor:
	def __init__(self, width, height, inter=cv2.INTER_AREA):
		# store the target image width, height, and interpolation
		# method used when resizing
		self.width = width
		self.height = height
		self.inter = inter
	def preprocess(self, image):
		# resize the image to a fixed size, ignoring the aspect
		# ratio
		return cv2.resize(image, (self.width, self.height),
			interpolation=self.inter)

In [3]:
class SimplePreprocessorGray:

	def preprocess(self, image):
		return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY )

In [6]:
imagePath = './argent06.jpg'
sp = Preprocessor(32, 32)
spg = SimplePreprocessorGray()
preprocessors = [spg]
image = cv2.imread(imagePath)

sift = cv2.SIFT_create()
principais_caracteristicas = []
principais_labels = []
n_sample = 50

keypoints, descriptors = sift.detectAndCompute(image, None)

principais = descriptors[np.random.randint(descriptors.shape[0], size=n_sample)]

principais_caracteristicas[(n_sample):(n_sample)+n_sample] = principais[:]

      # check to see if our preprocessors are not None
if preprocessors is not None:
  # loop over the preprocessors and apply each to
  # the image
  for p in preprocessors:
    image = p.preprocess(image)

  # cv2.imshow('bla', image)
  # cv2.waitKey(0)   

  # cv2.destroyAllWindows(

  
descriptors

array([[ 5.,  6., 13., ...,  5.,  5., 20.],
       [40.,  7.,  5., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ..., 21., 30., 39.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  6.],
       [13., 71., 56., ...,  0.,  0.,  0.],
       [38., 13.,  7., ...,  0.,  0.,  3.]], dtype=float32)

In [747]:
# Definindo os parametros
class Parametros:
  dataset = "./Imagens"
  neighbors = 1
  jobs = -1

args = vars(Parametros)

In [748]:
# grab the list of images that we'll be describing
print("[INFO] loading images...")
imagePaths = list(paths.list_images(args["dataset"]))
# initialize the image preprocessor, load the dataset from disk,
# and reshape the data matrix
sp = Preprocessor(32, 32)
spg = SimplePreprocessorGray()
sdl = SimpleDatasetLoader(preprocessors=[spg])
(data, labels) = sdl.load(imagePaths, verbose=500)
#data = data.reshape((data.shape[0], 60200))

# show some information on memory consumption of the images
print("[INFO] features matrix: {:.1f}MB".format(
	data.nbytes / (1024 * 1024.0)))

[INFO] loading images...
[INFO] features matrix: 2.5MB


In [749]:
# Obtendo keypoints das imagens
sift = cv2.SIFT_create()
principais_caracteristicas = []
principais_labels = []
n_sample = 50 # 50 keypoints de cada img pra tbm nao ficar pesado

# For cada imagem
for idx, img in enumerate(data):
  # pega o sift de cada img
  keypoints, descriptors = sift.detectAndCompute(data[idx], None)

  principais = descriptors[np.random.randint(descriptors.shape[0], size=n_sample)]

  principais_caracteristicas[(idx*n_sample):(idx*n_sample)+n_sample] = principais[:]

In [785]:
# Construindo o vocabulario
print(len(data)) # total de imgs
print(len(principais)) # 50 keypoints por imagem
print(len(principais_caracteristicas)) # total de imagens * keypoints por imagem

vocab_size = len(principais_caracteristicas) // vocab_size_percentage # Vocabulario vai ter {vocab_size_percentage}% das keypoints
vocab = KMeans(n_clusters=2).fit(principais_caracteristicas, vocab_size)

44
50
2200


C:\Users\castr\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [2]:
# Obtendo histogramas da base de imgs X vocabulario das principais keypoints

histo_list = []

# For cada imagem
for idx, img in enumerate(data):
  # Obtenho novamente os keypoints da img
  keypoints, descriptors = sift.detectAndCompute(data[idx], None)

  # Verifico como o vocabulario enquadra eles, montando assim o histograma
  histo = np.zeros(n_sample)
  nkp = np.size(keypoints)

  for d in descriptors:
      idx = vocab.predict([d])
      histo[idx] += 1/nkp # Because we need normalized histograms, I prefere to add 1/nkp directly

  histo_list.append(histo)

print("hist_list: ", len(histo_list))

NameError: name 'data' is not defined

In [787]:
# transforma as categorias em inteiros
le = LabelEncoder()
labels_number = le.fit_transform(labels)

trainX, testX, trainY, testY = train_test_split(histo_list, labels_number, test_size=test_size, random_state=42)

In [788]:
print(len(trainX))
print(len(trainY))

30
30


In [789]:
# Com esses histogramas, eu treino o meu modelo

knn_model = KNeighborsClassifier(n_neighbors=n_neighbors)
knn_model.fit(trainX, trainY)

KNeighborsClassifier(n_neighbors=3)

In [790]:
print(classification_report(testY, knn_model.predict(testX)))

              precision    recall  f1-score   support

           0       0.57      0.67      0.62         6
           1       0.71      0.62      0.67         8

    accuracy                           0.64        14
   macro avg       0.64      0.65      0.64        14
weighted avg       0.65      0.64      0.64        14



In [756]:
# Exportando
import pickle

pickle.dump(vocab, open("./ml_models/MontadorHistograma_v1.pkl", "wb"))
pickle.dump(knn_model, open("./ml_models/LeitorHistograma_v1.pkl", "wb"))